Import Libraries

In [ ]:
import pandas as pd
import re
import torch
from nltk.tokenize import word_tokenize
import random
from multiprocessing import Pool
import pickle
from tqdm import tqdm

Load file and extract a 500 million character subset

In [ ]:
file = open("notes.txt","r") 
text = file.read()

file.close() 

text = text[:5000000000]

Locate the PHI in the text, and make a separate string where PHI caracters are replaced by $

In [ ]:
temp = ""
phi = False
i = 0
while i < len(text):
    if i + 3 >= len(text):
        temp+=text[i:]
        break
    snip = text[i:i+3]
    if snip == "[**":
        phi = True
    if phi:
        if snip == "**]":
            temp+="$$$"
            phi = False
            i+=3
        else:
            temp+="$"
            i+=1
        continue
    temp+=text[i]
    i+=1

Use the PHI reference string created in the previous step to separate PHI and no-PHI.
The result is a list where each element is a string, which is either all PHI nor non-PHI. At the same time we create a labels list which labels each element as 1 (phi) or 0 (non-phi)

In [ ]:
print("separating phi and non phi")
step = 1
all_listed = []
i = 0
labels = []
while i < len(text)-1:
    temp_phi_str = ""
    temp_other_str = ""
    if temp[i] == "$":
        for j in range(i,len(text)):
            if temp[j] == "$":
                temp_phi_str+=text[j]
            else:
                break
        step = len(temp_phi_str)
        i+=step
        all_listed.append(temp_phi_str)
        labels.append(1)
    else:
        for j in range(i,len(text)):
            if temp[j] != "$":
                temp_other_str += text[j]
            else:
                break
        step = len(temp_other_str)
        i+=step
        all_listed.append(temp_other_str)
        labels.append(0)
    step = 1

Each of the elements of the list created in the previous step is split on whitespace and stripped of trailing and leading whitespaces.
The resulting words are again appendied to another list in order, and the corresponding labels are expanded appropriately.

In [ ]:
split_labels_list = []
split_word_list = []
    
for i in range(len(all_listed)):
    if labels[i] == 0:
        all_listed[i] = " ".join(all_listed[i].split())
        temp_split_list = all_listed[i].split(" ")
        split_word_list.extend(temp_split_list)
        split_labels_list.extend([labels[i]]*len(temp_split_list))
    else:
        split_word_list.append(all_listed[i][3:-3])
        split_labels_list.append(labels[i])


Different tests/rules, based on which the labels are expanded

In [ ]:
def isdate(string):
    pattern = re.compile("([0-9]|[0-9]{2}|[0-9]{4})-([0-9]|[0-9]{2}|[0-9]{4})-?([0-9]|[0-9]{2}|[0-9]{4})?$")
    return bool(re.match(pattern,string))


#def isage(string):
#    if string in age_phi_lis:
#        return True
#    else:
#        return False
    
    
def get_finer_labels(orig_string,orig_label):
    label = orig_label
    string = orig_string.lower()
    string = " ".join(string.split())
    if "name" in string:
        label = "NAME"
    elif "location" in string or "apart" in string or "address" in string or "country" in string:
        label= "LOC"
    elif "hospital" in string:
        label= "HOSP"
    elif isdate(string):
        label = "DATE"
    elif "Month/Day" in orig_string:
        label = "MNTH/DAY"
    elif "telephone" in string or "fax" in string:
        label = "CONTACT"
    elif "number" in string or "identif"in string :
        label = "UNIQID"
    #elif isage(orig_string):
    #    label = "AGE"
    return label

The actual expansion of labels is done here

In [ ]:
finer_labels = []
for i in range(len(split_word_list)):
    finer_labels.append(get_finer_labels(split_word_list[i],split_labels_list[i]))

Re-identification of data i.e. replacing vague place holder with realistic lookoing data

In [ ]:
#Random names, addresses and hospiytals are stored in these files
file = open("first_names_female.txt","r") 
first_names_f = (file.read()).split("\n")  
file.close()

file = open("first_names_male.txt","r") 
first_names_m = (file.read()).split("\n")  
file.close()

file = open("last_names.txt","r") 
last_names = (file.read()).split("\n")  
file.close()

file = open("sampled_hospitals.txt","r") 
hospitals = (file.read()).split("\n")  
file.close()

In [ ]:
#Foe dates
def get_random_date():
    month = str(random.sample(range(1,12),1)[0])
    if month in [1,3,5,7,8,10,12]:
        day = str(random.sample(range(1,31),1)[0])
    elif month==2:
        day = str(random.sample(range(1,28),1)[0])
    else:
        day = str(random.sample(range(1,30),1)[0])
    return month+"-"+day

In [ ]:
#hospitals
def get_random_hosp():
    return random.choice(hospitals)

#names
def get_random_name(string):
    string = string.lower()
    first = random.choice([random.choice(first_names_m),random.choice(first_names_f)])
    last = random.choice(last_names)
    if "first" in string:
        return first
    elif "last" in string:
        return last
    else:
        return first + " " + last

    
from random import randint

#random numbers
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

    
from numpy.random import choice
def get_random_phone_fax_number():
    nums = [str(random_with_N_digits(3))+"-"+str(random_with_N_digits(3))+"-"+str(random_with_N_digits(4)) for x in range(10)]
    nums.append(str(random_with_N_digits(10)))
    return random.choice(nums)

#unique ids
def get_uniqid():
    nums = [str(random_with_N_digits(x)) for x in [4,6,10,12,16]]
    return random.choice(nums)

Actual filling in of data is done here

In [ ]:
print("filling in reid")

for i in tqdm(range(len(split_word_list))):
    if finer_labels[i] == "MNTH/DAY":
        rand_date = get_random_date()
        finer_labels[i] = "DATE"
        split_word_list[i] = rand_date
    elif finer_labels[i] == "HOSP":
        rand_hosp = get_random_hosp()
        split_word_list[i] = rand_hosp
        finer_labels[i] = "LOC"
    elif finer_labels[i] == "NAME":
        split_word_list[i] = get_random_name(split_word_list[i])
    elif finer_labels[i] == "CONTACT":
        split_word_list[i] = get_random_phone_fax_number()
    elif finer_labels[i] == "UNIQID":
        split_word_list[i] = get_uniqid()
    elif finer_labels[i] == "AGE":
        split_word_list[i] = str(random.choice(range(2,90)))

Once the PHI labels are replaced by finer labels, we label everything non-PHI

In [ ]:
for i in range(len(split_word_list)):
    split_word_list[i] = " ".join(split_word_list[i].split()).lower()
    if finer_labels[i] == 1 or finer_labels[i] == 0:
        finer_labels[i] = "OTHER"


Using NLTK word tokenize, we tokenize phi and non-phi string while expanding the corresponding labels

In [ ]:
tokenized_word_list = []
tokenized_label_list = []
for i in tqdm(range(len(split_word_list))):
        temp_split_list = word_tokenize(split_word_list[i])
        tokenized_word_list.extend(temp_split_list)
        tokenized_label_list.extend([finer_labels[i]]*len(temp_split_list))

To take care of date data, because dates are split by word tokeinze on hyphens which are then removed.

In [ ]:
tokenized_word_list2 = []
tokenized_label_list2 = []
for i in tqdm(range(len(tokenized_word_list))):
        temp_split_list = tokenized_word_list[i].split("-")
        if len(temp_split_list) == 1:
            tokenized_word_list2.append(tokenized_word_list[i])
            tokenized_label_list2.append(tokenized_label_list[i])
        elif len(temp_split_list) == 2:
            tokenized_word_list2.extend([temp_split_list[0],"-",temp_split_list[1]])
            tokenized_label_list2.extend([tokenized_label_list[i]]*3)
        elif len(temp_split_list) == 3:
            tokenized_word_list2.extend([temp_split_list[0],"-",temp_split_list[1],"-",temp_split_list[2]])
            tokenized_label_list2.extend([tokenized_label_list[i]]*5)

In [ ]:
tokenized_word_list = []
tokenized_label_list =  []

for i in tqdm(range(len(tokenized_word_list2))):
    if tokenized_word_list2[i] != "":
        tokenized_word_list.append(tokenized_word_list2[i])
        tokenized_label_list.append(tokenized_label_list2[i])

tokenized_word_list2 = []
tokenized_label_list2 = []

Making senetnces of length 8 which will be input to the BERT model

In [ ]:
def get_sentences(word_list, label_list, l):
    sentences = []
    sentence_labels = []
    num_of_words = len(word_list)
    for i in tqdm(range(0,num_of_words,l)):
        if i+l < num_of_words:
            sentences.append(word_list[i:i+l])
            sentence_labels.append(label_list[i:i+l])
        else:
            sentences.append(word_list[i:])
            sentence_labels.append(label_list[i:])
    return sentences, sentence_labels
    
l = 8
sentences, sentence_labels = get_sentences(tokenized_word_list, tokenized_label_list, l)

Serializing the preprocessed data

In [ ]:
pickle.dump(sentences, open( "sentences_8.p", "wb" ))
pickle.dump(sentence_labels, open( "sentence_labels_8.p", "wb" ))